In [1]:
def WriteHeader(f, TaskName):
    f.write("#!/bin/bash\n\n")
    f.write("#SBATCH --partition=single\n")
    f.write("#SBATCH --ntasks-per-node=40\n")
    f.write("#SBATCH --time=72:00:00\n")
    f.write(f"#SBATCH --job-name={TaskName}\n")
    f.write("#SBATCH --error=%x.%j.err\n")
    f.write("#SBATCH --output=%x.%j.out\n")
    f.write("#SBATCH --mail-user=hzhao@teco.edu\n\n")

In [6]:
import os
import torch
from configuration import *
from utils import *
import copy
args = parser.parse_args([])

files = []
if os.path.exists('./FaultAnalysisDropout/models'):
    files = os.listdir('./FaultAnalysisDropout/models')
files = [f for f in files if f.endswith('.model')]
files.sort()

def write_jobs(f, job_counter, epsilon, ds, seed, do):
    line = f'python3 experiment.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --dropout {do} --projectname FaultAnalysisDropout &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

f = open(f'run_experiment_{file_counter:02d}.sh', 'w')
WriteHeader(f, f'FAD')

for epsilon in [0., 0.05, 0.1]:
    for do in [0., 0.05, 0.1]:
        for ds in range(13):
            for seed in range(10):
                if job_counter >= 10:
                    f.write("\nwait\n")
                    f.close()
                    file_counter += 1
                    job_counter = 0
                    
                    f = open(f'run_experiment_{file_counter:02d}.sh', 'w')
                    WriteHeader(f, f'FAD')

                local_args = copy.deepcopy(args)
                local_args.DATASET = ds
                local_args.e_train = epsilon
                local_args.dropout = do
                local_args.SEED = seed
                local_args = FormulateArgs(local_args)
                _, datainfo = GetDataLoader(local_args, 'train')
                data = f"data_{local_args.DATASET:02d}_{datainfo['dataname']}_seed_{local_args.SEED:02d}_epsilon_{local_args.e_train}_dropout_{local_args.dropout}.model"
                
                if data not in files:
                    job_counter = write_jobs(f, job_counter, epsilon, ds, seed, do)

if not f.closed:
    f.write("\nwait\n")
    f.close()

In [2]:
def write_jobs(f, job_counter, epsilon, ds, seed, do):
    line = f'python3 evaluation.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --dropout {do} &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

# Open the first file for the current epsilon
f = open(f'evaluation_{file_counter:02d}.sh', 'w')
WriteHeader(f, f'FADE')

for epsilon in [0., 0.1]:
    for do in [0., 0.1]:
        for ds in range(13):
            for seed in range(10):
                if job_counter >= 40:
                    f.write("\nwait\n")
                    f.close()
                    file_counter += 1
                    job_counter = 0
                    
                    f = open(f'evaluation_{file_counter:02d}.sh', 'w')
                    WriteHeader(f, f'FADE')

                job_counter = write_jobs(f, job_counter, epsilon, ds, seed, do)

if not f.closed:
    f.write("\nwait\n")
    f.close()

In [11]:
f = open(f'run_experiment.sh', 'w')
for i in range(1,37):
    f.write(f'sbatch run_experiment_{i:02d}.sh \n')
f.close()

In [3]:
f = open(f'run_evaluation.sh', 'w')
for i in range(1,13):
    f.write(f'sbatch evaluation_{i:02d}.sh \n')
f.close()